In [1]:
from jupytertracerviz import init_multigpus_repl, multigpus

init_multigpus_repl()

/root/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install

In [2]:
%%multigpus

import torch

torch._dynamo.config.capture_scalar_outputs = True

import os
import math
import time
from torch import nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
from functools import partial
from torch.distributed.algorithms._checkpoint.checkpoint_wrapper import (
    CheckpointImpl,
    apply_activation_checkpointing,
    checkpoint_wrapper,
)
from torch import distributed as dist
from torch.distributed.tensor import distribute_tensor
from torch.distributed.device_mesh import init_device_mesh
from torch.utils.data.distributed import DistributedSampler
from torch.distributed.fsdp import fully_shard, MixedPrecisionPolicy, CPUOffloadPolicy
from transformers import (
    set_seed,
    get_linear_schedule_with_warmup,
    AutoConfig,
    AutoTokenizer,
    Glm4MoeForCausalLM,
)
from transformers.models.glm4_moe.modeling_glm4_moe import (
    Glm4MoeMLP,
    Glm4MoeDecoderLayer,
    Glm4MoeTopkRouter,
    ACT2FN,
)
from transformers.models.glm4_moe import modeling_glm4_moe
from liger_kernel.transformers import LigerFusedLinearCrossEntropyLoss
from streaming import LocalDataset
from streaming.base.format.mds.encodings import Encoding, _encodings
from tqdm import tqdm
import numpy as np
import wandb

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

class Dataset(Dataset):
    def __init__(self, folder, sequence_length=16384):
        self.dataset = LocalDataset(local=folder)
        self.sequence_length = sequence_length
    
    def __getitem__(self, idx):
        data = self.dataset[idx]
        data.pop('audio', None)
        data.pop('text', None)
        data.pop('token_type_ids', None)

        for k in data.keys():
            data[k] = data[k].astype(np.int64)

        data['labels'] = data['input_ids'].copy()
        attention_mask_sum = data['attention_mask'].sum()
        
        if attention_mask_sum < self.sequence_length:
            balance = self.sequence_length - attention_mask_sum
            data['input_ids'] = np.concatenate([data['input_ids'], np.array([151329] * balance)])
            data['position_ids'] = np.concatenate([data['position_ids'], np.array([0] * balance)])
            data['labels'] = np.concatenate([data['labels'], np.array([-100] * balance)])
            data['attention_mask'] = np.concatenate([data['attention_mask'], np.array([balance])])
    
        return data
    
    def __len__(self):
        return len(self.dataset)

def collator(batch):
    batch = [b for b in batch if b is not None]
    input_ids = [b['input_ids'] for b in batch]
    position_ids = [b['position_ids'] for b in batch]
    labels = [b['labels'] for b in batch]
    attention_mask = [b['attention_mask'] for b in batch]
    input_ids = np.concatenate(input_ids)
    position_ids = np.concatenate(position_ids)
    labels = np.concatenate(labels)
    query_lens = np.concatenate(attention_mask)
    cumsum = [0] + np.cumsum(query_lens).tolist()
    max_cumsum = int(np.max(cumsum))
    cu_seq_lens_q = torch.tensor(cumsum, dtype=torch.int32)
    cu_seq_lens_k = torch.tensor(cumsum, dtype=torch.int32)
    max_seqlen_q = int(np.max(query_lens))
    return {
        'input_ids': torch.tensor(input_ids)[None],
        'position_ids': torch.tensor(position_ids)[None],
        'labels': torch.tensor(labels)[None],
        'cu_seq_lens_q': cu_seq_lens_q,
        'cu_seq_lens_k': cu_seq_lens_k,
        'max_length_q': max_seqlen_q,
        'max_length_k': max_seqlen_q
    }

class Model(Glm4MoeForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.loss = LigerFusedLinearCrossEntropyLoss(reduction="sum")
        
    def forward(
        self, 
        input_ids, 
        attention_mask=None, 
        position_ids=None, 
        labels=None, 
        num_items_in_batch=None, 
        logits_to_keep=0,
        output_router_logits=None,
        **kwargs,
    ):
        super_out = self.model.forward(
            input_ids = input_ids,
            position_ids = position_ids, 
            attention_mask = attention_mask, 
            output_router_logits=output_router_logits,
            **kwargs,
        )
        if labels is not None:
            embeddings = super_out.last_hidden_state
            slice_indices = slice(-logits_to_keep, None) if isinstance(logits_to_keep, int) else logits_to_keep
            embeddings = embeddings[:, slice_indices, :]
            embeddings = embeddings[:,:-1].reshape(-1, embeddings.shape[-1])
            labels = labels[..., 1:].contiguous()
            labels = labels.reshape(-1)
            
            loss = self.loss(self.lm_head.weight, embeddings, labels)
            num_items_in_batch = num_items_in_batch.to(loss.device)

            loss = loss / num_items_in_batch
            return {'loss': loss}
        return super_out

class ExpertLoRAWeights(nn.Module):
    """Wrapper to make expert LoRA weights more FSDP-friendly"""
    def __init__(self, num_experts, in_dim, out_dim, r, dtype=torch.bfloat16):
        super().__init__()
        self.A = nn.Parameter(torch.zeros(num_experts, in_dim, r, dtype=dtype))
        self.B = nn.Parameter(torch.zeros(num_experts, r, out_dim, dtype=dtype))
        
        with torch.no_grad():
            init.kaiming_uniform_(self.A, a=math.sqrt(5))
            
class Glm4MoeMoEExpertParallel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_experts = config.n_routed_experts
        self.top_k = config.num_experts_per_tok
        self.norm_topk_prob = config.norm_topk_prob

        self.gate_proj = nn.Parameter(torch.zeros(self.num_experts, config.hidden_size, config.moe_intermediate_size))
        self.up_proj = nn.Parameter(torch.zeros(self.num_experts, config.hidden_size, config.moe_intermediate_size))
        self.down_proj = nn.Parameter(torch.zeros(self.num_experts, config.moe_intermediate_size, config.hidden_size))
        self._is_stacked = False
        self.act_fn = ACT2FN[config.hidden_act]

        self.gate = Glm4MoeTopkRouter(config)
        self.shared_experts = Glm4MoeMLP(
            config=config, intermediate_size=config.moe_intermediate_size * config.n_shared_experts
        )
        self.gate_lora = None
        self.up_lora = None
        self.down_lora = None
    
    def apply_lora_stack(self, r, alpha):
        if self._is_stacked:
            return

        self.r = r
        self.alpha = alpha
        self.alpha = alpha / r
        
        self._is_stacked = True

        self.gate_lora = ExpertLoRAWeights(
            self.num_experts, self.gate_proj.shape[1], self.gate_proj.shape[2], r
        )
        self.up_lora = ExpertLoRAWeights(
            self.num_experts, self.up_proj.shape[1], self.up_proj.shape[2], r
        )
        self.down_lora = ExpertLoRAWeights(
            self.num_experts, self.down_proj.shape[1], self.down_proj.shape[2], r
        )

    def moe(self, hidden_states: torch.Tensor, topk_indices: torch.Tensor, topk_weights: torch.Tensor):
        inputs = hidden_states
        M = hidden_states.shape[0]
        hidden_dim = hidden_states.shape[-1]

        sort_indices = topk_indices.view(-1).argsort()  # (M * topk,)
        sorted_pos = sort_indices // self.top_k
        grouped_inputs = inputs[sorted_pos]  # (M * topk, dim)

        experts_count = topk_indices.view(-1).bincount(minlength=self.num_experts)
        cu_experts_count = experts_count.cumsum(dim=0).to(torch.int32)

        gate_out = torch._grouped_mm(
            grouped_inputs,
            self.gate_proj,
            cu_experts_count,
        )
        if self.gate_lora is not None:
            gate_out_lora_A = torch._grouped_mm(
                grouped_inputs,
                self.gate_lora.A,
                cu_experts_count,
            )
            gate_out_lora_B = torch._grouped_mm(
                gate_out_lora_A,
                self.gate_lora.B,
                cu_experts_count,
            )
            gate_out = gate_out + gate_out_lora_B * self.alpha
        
        up_out = torch._grouped_mm(
            grouped_inputs,
            self.up_proj,
            cu_experts_count,
        )
        if self.up_lora is not None:
            up_out_lora_A = torch._grouped_mm(
                grouped_inputs,
                self.up_lora.A,
                cu_experts_count,
            )
            up_out_lora_B = torch._grouped_mm(
                up_out_lora_A,
                self.up_lora.B,
                cu_experts_count,
            )
            up_out = up_out + up_out_lora_B * self.alpha

        intermediate = self.act_fn(gate_out) * up_out
        
        down_out = torch._grouped_mm(
            intermediate,
            self.down_proj,
            cu_experts_count,
        )

        if self.down_lora is not None:
            down_out_lora_A = torch._grouped_mm(
                intermediate,
                self.down_lora.A,
                cu_experts_count,
            )
            down_out_lora_B = torch._grouped_mm(
                down_out_lora_A,
                self.down_lora.B,
                cu_experts_count,
            )
            down_out = down_out + down_out_lora_B * self.alpha

        down_out = down_out * topk_weights.view(-1)[sort_indices].unsqueeze(-1)

        outputs = inputs.new_zeros(M, hidden_dim)
        sorted_pos_expanded = sorted_pos.unsqueeze(-1).expand(-1, hidden_dim)
        outputs.scatter_add_(0, sorted_pos_expanded, down_out.to(outputs.dtype))

        return outputs
    
    def forward(self, hidden_states):
        residuals = hidden_states
        orig_shape = hidden_states.shape
        topk_indices, topk_weights = self.gate(hidden_states)
        hidden_states = hidden_states.view(-1, hidden_states.shape[-1])
        hidden_states = self.moe(hidden_states, topk_indices, topk_weights).view(*orig_shape)
        hidden_states = hidden_states + self.shared_experts(residuals)
        return hidden_states

modeling_glm4_moe.Glm4MoeMoE = Glm4MoeMoEExpertParallel

class LinearLoRA(nn.Module):
    def __init__(self, linear: nn.Linear, r=4, alpha=1.0):
        super().__init__()
        self.linear = linear
        self.scaling = alpha / r

        in_features = linear.in_features
        out_features = linear.out_features
        
        self.lora_A = nn.Parameter(torch.zeros(r, in_features, dtype=torch.bfloat16))
        self.lora_B = nn.Parameter(torch.zeros(out_features, r, dtype=torch.bfloat16))
        
        with torch.no_grad():
            init.kaiming_uniform_(self.lora_A, a=math.sqrt(5))
            # lora_B stays zero

    def forward(self, x):
        out = self.linear(x)
        lora_out = F.linear(F.linear(x.to(self.lora_A.dtype), self.lora_A), self.lora_B) * self.scaling
        return out + lora_out.to(out.dtype)
        
def check_fn(module):
    return isinstance(module, (Glm4MoeDecoderLayer, Glm4MoeMoEExpertParallel))

non_reentrant_wrapper = partial(
    checkpoint_wrapper,
    checkpoint_impl=CheckpointImpl.NO_REENTRANT,
)

rank = int(os.environ["LOCAL_RANK"])
world_size = int(os.environ['WORLD_SIZE'])
device_type = torch.accelerator.current_accelerator()
device = torch.device(f"{device_type}:{rank}")
device_mesh = init_device_mesh(device_type.type, (world_size,), mesh_dim_names=("dp",))
dp_mesh = device_mesh["dp"]
dp_rank = dp_mesh.get_local_rank()
dp_world_size = dp_mesh.size()

In [3]:
%%multigpus

model_name = "nfs/nfs/GLM-4.5-Air"
model = Model.from_pretrained(
    model_name, 
    attn_implementation="kernels-community/vllm-flash-attn3",
    torch_dtype=torch.bfloat16,
)

for name, param in model.named_parameters():
    param.requires_grad = False

selected = [
    "q_proj", 
    "k_proj", 
    "v_proj", 
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj"
]

rank_lora = 256
alpha_lora = 512

for name, module in tqdm(model.named_modules()):
    for child_name, child in module.named_children():
        if len(child_name) and any([a in child_name for a in selected]) and isinstance(child, nn.Linear):
            
            if 'mlp.experts' in name:
                continue

            lora = LinearLoRA(child, r=rank_lora, alpha=alpha_lora)
            setattr(module, child_name, lora)

top_k = model.config.num_experts_per_tok
r = rank_lora // top_k
alpha = alpha_lora // top_k

for module in tqdm(model.modules()):
    if isinstance(module, Glm4MoeMoEExpertParallel):
        module.apply_lora_stack(r=r, alpha=alpha)

In [4]:
%%multigpus

# lora_mp_policy = MixedPrecisionPolicy(
#     param_dtype=torch.float32,
#     reduce_dtype=torch.float32,
# )

# for name, module in tqdm(model.named_modules()):
#     for child_name, child in module.named_children():
#         if '.lora' in name:
#             fully_shard(child, mp_policy=lora_mp_policy)

fsdp_kwargs = {}
fsdp_kwargs["mp_policy"] = MixedPrecisionPolicy(
    param_dtype=torch.bfloat16,
    reduce_dtype=torch.float32,
)

for module in tqdm(model.modules()):
    if isinstance(module, Glm4MoeDecoderLayer):
        fully_shard(module, **fsdp_kwargs)
fully_shard(model, **fsdp_kwargs)

In [5]:
%%multigpus

apply_activation_checkpointing(
    model,
    checkpoint_wrapper_fn=non_reentrant_wrapper,
    check_fn=check_fn,
)

In [6]:
%%multigpus

model = torch.compile(model)

In [9]:
# %%multigpus

# if rank == 0:
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             print(name, param.size(), param.dtype)

In [10]:
# %%multigpus

# dataset = Dataset('multipacking-glm')

# b = [dataset[0], dataset[1]]
# b = collator(b)
# with torch.no_grad():
#     for k in b.keys():
#         if isinstance(b[k], torch.Tensor):
#             b[k] = b[k].to(model.device, non_blocking=True)

#     valid_tokens = (b['labels'] != -100).sum().item() * dp_world_size
#     b['num_items_in_batch'] = torch.tensor(valid_tokens)
#     out = model(**b, use_cache=False)

# out

In [11]:
%%multigpus

optim = torch.optim.AdamW(model.parameters(), lr=1e-4, fused=True, weight_decay=0.01)

In [12]:
%%multigpus

dataset = Dataset('multipacking-glm')
sampler = DistributedSampler(
    dataset,
    num_replicas=dp_world_size,
    rank=dp_rank,
    shuffle=True,
)
train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    num_workers=5,
    prefetch_factor=5,
    pin_memory=True,
    collate_fn=collator,
)
iter_train_loader = iter(train_loader)

In [ ]:
%%multigpus

@torch.no_grad()
def clip_grad_norm_(parameters, max_norm, norm_type=2.0):
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    else:
        parameters = list(parameters)
    
    grads = [p.grad for p in parameters if p.grad is not None]
    
    norms = []
    for grad in grads:
        if hasattr(grad, 'full_tensor'):  # DTensor
            grad_full = grad.full_tensor()
        else:
            grad_full = grad
        
        if norm_type == float('inf'):
            norms.append(grad_full.abs().max())
        else:
            norms.append(grad_full.norm(norm_type))
    
    if len(norms) == 0:
        return torch.tensor(0.0)
    
    total_norm = torch.stack(norms).norm(norm_type)
    
    clip_coef = max_norm / (total_norm + 1e-6)
    clip_coef_clamped = torch.clamp(clip_coef, max=1.0)
    
    for grad in grads:
        if hasattr(grad, 'full_tensor'):  # DTensor
            grad.mul_(clip_coef_clamped)
        else:
            grad.mul_(clip_coef_clamped)
    
    return total_norm
    
for i in range(5):
    total_tokens = 0
    b = next(iter_train_loader)
    for k in b.keys():
        if isinstance(b[k], torch.Tensor):
            b[k] = b[k].to(device, non_blocking=True)

    valid_tokens = (b['labels'] != -100).sum().item()
    total_tokens += valid_tokens
    token_tensor = torch.tensor([total_tokens], dtype=torch.long, device=device)
    dp_group = dp_mesh.get_group()
    dist.all_reduce(token_tensor, op=dist.ReduceOp.SUM, group=dp_group)
    global_total_tokens = token_tensor.item()
    b['num_items_in_batch'] = torch.tensor(global_total_tokens)
    out = model(**b, use_cache=False)
    loss = out["loss"] * dp_world_size
    print('done forward')
    loss.backward()
    print('done backward')
    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optim.step()
    optim.zero_grad()
    print('done grad')

    print(i, loss, out["loss"])

[GPU 3] done forward
[GPU 2] done forward
[GPU 5] done forward
[GPU 1] done forward
[GPU 0] done forward
[GPU 6] done forward
[GPU 4] done forward
[GPU 7] done forward
[GPU 6] done backward
[GPU 7] done backward
[GPU 1] done backward
[GPU 3] done backward
[GPU 0] done backward
[GPU 2] done backward
[GPU 4] done backward
[GPU 5] done backward
[GPU 7] done grad
[GPU 3] done grad
[GPU 5] done grad
[GPU 6] done grad
[GPU 2] done grad
[GPU 0] done grad
[GPU 4] done grad
[GPU 7] 0 tensor(4.0931, device='cuda:7', grad_fn=<MulBackward0>) tensor(0.5116, device='cuda:7', grad_fn=<CompiledFunctionBackward>)
[GPU 1] done grad
[GPU 6] 0 tensor(4.0936, device='cuda:6', grad_fn=<MulBackward0>) tensor(0.5117, device='cuda:6', grad_fn=<CompiledFunctionBackward>)
[GPU 5] 0 tensor(4.0933, device='cuda:5', grad_fn=<MulBackward0>) tensor(0.5117, device='cuda:5', grad_fn=<CompiledFunctionBackward>)
[GPU 2] 0 tensor(4.0941, device='cuda:2', grad_fn=<MulBackward0>) tensor(0.5118, device='cuda:2', grad_fn=<Com